**Installing and starting Ray**



In [1]:
pip install ray

     |████████████████████████████████| 54.7 MB 35 kB/s 
     |████████████████████████████████| 72 kB 599 kB/s 


**Installing XGBoost-Ray**

In [2]:
pip install xgboost_ray

     |████████████████████████████████| 136 kB 7.9 MB/s 


**Installing SKLearn**

In [3]:
pip install sklearn

**Simple training example**

In [4]:
from xgboost_ray import RayDMatrix, RayParams, train
from sklearn.datasets import load_breast_cancer

train_x, train_y = load_breast_cancer(return_X_y=True)
train_set = RayDMatrix(train_x, train_y)

evals_result = {}
bst = train(
    {
        "objective": "binary:logistic",
        "eval_metric": ["logloss", "error"],
    },
    train_set,
    evals_result=evals_result,
    evals=[(train_set, "train")],
    verbose_eval=False,
    ray_params=RayParams(num_actors=2, cpus_per_actor=1))

bst.save_model("model.xgb")
print("Final training error: {:.4f}".format(
    evals_result["train"]["error"][-1]))

/usr/local/lib/python3.7/dist-packages/xgboost_ray/main.py:166: UserWarning: You are using `xgboost_ray` with a legacy XGBoost version (version 0.90). While we try to support older XGBoost versions, please note that this library is only fully tested and supported for XGBoost >= 1.4. Please consider upgrading your XGBoost version (`pip install -U xgboost`).
  warnings.warn(LEGACY_WARNING)
2021-11-13 01:27:34,253	INFO main.py:971 -- [RayXGBoost] Created 2 new actors (2 total actors). Waiting until actors are ready for training.
2021-11-13 01:27:36,389	INFO main.py:1016 -- [RayXGBoost] Starting XGBoost training.
(pid=284) [01:27:36] WARNING: /workspace/src/learner.cc:622: Tree method is automatically selected to be 'approx' for distributed training.
(pid=284) [01:27:36] Tree method is automatically selected to be 'approx' for distributed training.
(pid=308) [01:27:36] WARNING: /workspace/src/learner.cc:622: Tree method is automatically selected to be 'approx' for distributed training.
(pi

Final training error: 0.0053


**Simple prediction example**

In [5]:
from xgboost_ray import RayDMatrix, RayParams, predict
from sklearn.datasets import load_breast_cancer
import xgboost as xgb

data, labels = load_breast_cancer(return_X_y=True)

dpred = RayDMatrix(data, labels)

bst = xgb.Booster(model_file="model.xgb")
pred_ray = predict(bst, dpred, ray_params=RayParams(num_actors=2))

print(pred_ray)

/usr/local/lib/python3.7/dist-packages/xgboost_ray/main.py:166: UserWarning: You are using `xgboost_ray` with a legacy XGBoost version (version 0.90). While we try to support older XGBoost versions, please note that this library is only fully tested and supported for XGBoost >= 1.4. Please consider upgrading your XGBoost version (`pip install -U xgboost`).
  warnings.warn(LEGACY_WARNING)
2021-11-13 01:27:38,052	INFO main.py:1535 -- [RayXGBoost] Created 2 remote actors.
2021-11-13 01:27:40,396	INFO main.py:1552 -- [RayXGBoost] Starting XGBoost prediction.


[0.07511145 0.04715708 0.02642454 0.11464816 0.08779665 0.19609766
 0.02642454 0.03200296 0.03968306 0.06930447 0.09290799 0.02642454
 0.03476055 0.05490047 0.05012454 0.02642454 0.03337562 0.02642454
 0.02642454 0.9528658  0.9751126  0.9751126  0.07511145 0.02642454
 0.02642454 0.04102892 0.03118352 0.02642454 0.02642454 0.08779665
 0.02642454 0.03476055 0.02642454 0.02642454 0.02642454 0.02642454
 0.0665335  0.9690314  0.33619487 0.0696831  0.5729184  0.15662025
 0.03451282 0.03200296 0.12301622 0.04102892 0.9751126  0.04096693
 0.97315884 0.9234187  0.9751126  0.9751126  0.9751126  0.03059936
 0.09855346 0.9751126  0.02642454 0.03755146 0.9751126  0.97315884
 0.9751126  0.97315884 0.02642454 0.9751126  0.03120249 0.03120249
 0.97315884 0.9751126  0.8630876  0.9751126  0.02642454 0.9751126
 0.02642454 0.18486539 0.9751126  0.02642454 0.9684864  0.07511145
 0.04528174 0.9751126  0.97315884 0.8423582  0.02642454 0.02642454
 0.9751126  0.02642454 0.18525673 0.02642454 0.9610139  0.91992

**Hyperparameter optimization**

In [6]:
from xgboost_ray import RayDMatrix, RayParams, train
from sklearn.datasets import load_breast_cancer

num_actors = 1
num_cpus_per_actor = 1

ray_params = RayParams(
    num_actors=num_actors, cpus_per_actor=num_cpus_per_actor)

def train_model(config):
    train_x, train_y = load_breast_cancer(return_X_y=True)
    train_set = RayDMatrix(train_x, train_y)

    evals_result = {}
    bst = train(
        params=config,
        dtrain=train_set,
        evals_result=evals_result,
        evals=[(train_set, "train")],
        verbose_eval=False,
        ray_params=ray_params)
    bst.save_model("model.xgb")

from ray import tune

# Specify the hyperparameter search space.
config = {
    "tree_method": "approx",
    "objective": "binary:logistic",
    "eval_metric": ["logloss", "error"],
    "eta": tune.loguniform(1e-4, 1e-1),
    "subsample": tune.uniform(0.5, 1.0),
    "max_depth": tune.randint(1, 9)
}

# Make sure to use the `get_tune_resources` method to set the `resources_per_trial`
analysis = tune.run(
    train_model,
    config=config,
    metric="train-error",
    mode="min",
    num_samples=4,
    resources_per_trial=ray_params.get_tune_resources())
print("Best hyperparameters", analysis.best_config)

2021-11-13 01:27:40,509	WARNING function_runner.py:564 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.
2021-11-13 01:27:40,532	INFO logger.py:606 -- pip install 'ray[tune]' to see TensorBoard files.
2021-11-13 01:27:40,535	WARNING callback.py:115 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`


Trial name,status,loc,eta,max_depth,subsample
train_model_e486b_00000,PENDING,,0.04731,3,0.877711
train_model_e486b_00001,PENDING,,0.000144307,7,0.848184
train_model_e486b_00002,PENDING,,0.00120555,8,0.591216
train_model_e486b_00003,PENDING,,0.034596,4,0.754874


(pid=388) /usr/local/lib/python3.7/dist-packages/xgboost_ray/main.py:166: UserWarning: You are using `xgboost_ray` with a legacy XGBoost version (version 0.90). While we try to support older XGBoost versions, please note that this library is only fully tested and supported for XGBoost >= 1.4. Please consider upgrading your XGBoost version (`pip install -U xgboost`).
(pid=388)   warnings.warn(LEGACY_WARNING)
(pid=388) /usr/local/lib/python3.7/dist-packages/xgboost_ray/main.py:422: UserWarning: `num_actors` in `ray_params` is smaller than 2 (1). XGBoost will NOT be distributed!
(pid=388)   f"`num_actors` in `ray_params` is smaller than 2 "
(pid=388) 2021-11-13 01:27:42,860	INFO main.py:971 -- [RayXGBoost] Created 1 new actors (1 total actors). Waiting until actors are ready for training.
(pid=388) 2021-11-13 01:27:45,473	INFO main.py:1016 -- [RayXGBoost] Starting XGBoost training.


Trial name,status,loc,eta,max_depth,subsample
train_model_e486b_00000,RUNNING,172.28.0.2:388,0.04731,3,0.877711
train_model_e486b_00001,PENDING,,0.000144307,7,0.848184
train_model_e486b_00002,PENDING,,0.00120555,8,0.591216
train_model_e486b_00003,PENDING,,0.034596,4,0.754874


Result for train_model_e486b_00000:
  date: 2021-11-13_01-27-46
  done: false
  experiment_id: 0e877a910b2e4f169fb0d8259fdaa038
  hostname: 1a8220ffa18b
  iterations_since_restore: 1
  node_ip: 172.28.0.2
  pid: 388
  time_since_restore: 3.7011101245880127
  time_this_iter_s: 3.7011101245880127
  time_total_s: 3.7011101245880127
  timestamp: 1636766866
  timesteps_since_restore: 0
  train-error: 0.050967
  train-logloss: 0.654491
  training_iteration: 1
  trial_id: e486b_00000
  
Result for train_model_e486b_00000:
  date: 2021-11-13_01-27-46
  done: true
  experiment_id: 0e877a910b2e4f169fb0d8259fdaa038
  experiment_tag: 0_eta=0.04731,max_depth=3,subsample=0.87771
  hostname: 1a8220ffa18b
  iterations_since_restore: 10
  node_ip: 172.28.0.2
  pid: 388
  time_since_restore: 3.776052236557007
  time_this_iter_s: 0.006272792816162109
  time_total_s: 3.776052236557007
  timestamp: 1636766866
  timesteps_since_restore: 0
  train-error: 0.022847
  train-logloss: 0.41105
  training_iteration

...

Result for train_model_e486b_00003:
  date: 2021-11-13_01-28-03
  done: false
  experiment_id: 12d1cbb179f84a1fb1052a5af44f5976
  hostname: 1a8220ffa18b
  iterations_since_restore: 1
  node_ip: 172.28.0.2
  pid: 702
  time_since_restore: 3.710376739501953
  time_this_iter_s: 3.710376739501953
  time_total_s: 3.710376739501953
  timestamp: 1636766883
  timesteps_since_restore: 0
  train-error: 0.038664
  train-logloss: 0.664134
  training_iteration: 1
  trial_id: e486b_00003
  
Result for train_model_e486b_00003:
  date: 2021-11-13_01-28-03
  done: true
  experiment_id: 12d1cbb179f84a1fb1052a5af44f5976
  experiment_tag: 3_eta=0.034596,max_depth=4,subsample=0.75487
  hostname: 1a8220ffa18b
  iterations_since_restore: 10
  node_ip: 172.28.0.2
  pid: 702
  time_since_restore: 3.7723660469055176
  time_this_iter_s: 0.005841732025146484
  time_total_s: 3.7723660469055176
  timestamp: 1636766883
  timesteps_since_restore: 0
  train-error: 0.017575
  train-logloss: 0.465689
  training_iteratio

(pid=702) 2021-11-13 01:28:03,786	INFO main.py:1498 -- [RayXGBoost] Finished XGBoost training on training data with total N=569 in 3.79 seconds (1.12 pure XGBoost training time).


Trial name,status,loc,eta,max_depth,subsample,iter,total time (s),train-logloss,train-error
train_model_e486b_00000,TERMINATED,172.28.0.2:388,0.04731,3,0.877711,10,3.77605,0.41105,0.022847
train_model_e486b_00001,TERMINATED,172.28.0.2:494,0.000144307,7,0.848184,10,3.77883,0.691882,0.010545
train_model_e486b_00002,TERMINATED,172.28.0.2:598,0.00120555,8,0.591216,10,3.76759,0.683029,0.02812
train_model_e486b_00003,TERMINATED,172.28.0.2:702,0.034596,4,0.754874,10,3.77237,0.465689,0.017575


2021-11-13 01:28:03,963	INFO tune.py:630 -- Total run time: 23.46 seconds (23.27 seconds for the tuning loop).


Best hyperparameters {'tree_method': 'approx', 'objective': 'binary:logistic', 'eval_metric': ['logloss', 'error'], 'eta': 0.00014430696549007558, 'subsample': 0.848184273852242, 'max_depth': 7}
